# AlexNet — Architektúra

Súbor: alexnet.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

architektúra podľa: https://medium.com/datadriveninvestor/five-powerful-cnn-architectures-b939c9ddd57b

Import knižníc

In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 

Načítanie dát

In [3]:
train = np.load('../data/1500_train.npz')
test = np.load('../data/1500_test.npz')

In [4]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
print(x_train.shape)
print(x_test.shape)

(16000, 128, 128, 1)
(16000, 128, 128, 1)


In [7]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

CNN AlexNet

In [8]:
model = Sequential()

In [9]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [10]:
model.add(Conv2D(96, 11, strides=4, activation='relu'))

In [11]:
model.add(MaxPooling2D(3, 2))

In [12]:
model.add(Conv2D(256, 5, strides=1, padding='same', activation='relu'))

In [13]:
model.add(MaxPooling2D(3, 2))

In [14]:
model.add(Conv2D(384, 3, strides=1, padding='same', activation='relu'))

In [15]:
model.add(Conv2D(256, 3, strides=1, padding='same', activation='relu'))

In [16]:
model.add(MaxPooling2D(3, 2))

In [17]:
model.add(Flatten())

In [18]:
model.add(Dense(4096, activation='relu'))

In [19]:
model.add(Dropout(0.5))

In [20]:
model.add(Dense(4096, activation='relu'))

In [21]:
model.add(Dropout(0.5))

In [22]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [23]:
adam = Adam()

In [24]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [25]:
saved_model = "alexnet.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [26]:
f = model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 16000 samples, validate on 16000 samples
Epoch 1/15
16000/16000 [==============================] - 466s 29ms/step - loss: 0.4870 - accuracy: 0.7434 - val_loss: 0.3412 - val_accuracy: 0.8573

Epoch 00001: val_accuracy improved from -inf to 0.85731, saving model to alexnet.hdf5
Epoch 2/15
16000/16000 [==============================] - 390s 24ms/step - loss: 0.3312 - accuracy: 0.8539 - val_loss: 0.2761 - val_accuracy: 0.8928

Epoch 00002: val_accuracy improved from 0.85731 to 0.89275, saving model to alexnet.hdf5
Epoch 3/15
16000/16000 [==============================] - 541s 34ms/step - loss: 0.2578 - accuracy: 0.8949 - val_loss: 0.2598 - val_accuracy: 0.8979

Epoch 00003: val_accuracy improved from 0.89275 to 0.89794, saving model to alexnet.hdf5
Epoch 4/15
16000/16000 [==============================] - 681s 43ms/step - loss: 0.2090 - accuracy: 0.9178 - val_loss: 0.2132 - val_accuracy: 0.9213

Epoch 00004: val_accuracy improved from 0.89794 to 0.92131, saving model to alexnet.h

Načítanie modelu

In [27]:
print("Loading model....")
model = load_model('alexnet.hdf5')

Loading model....


Predikcia

In [28]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

In [29]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.20953427624795587
Test accuracy: 0.9480624794960022


Confusion Matrix

In [30]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [31]:
print(matrix)
# TN / FN
# FP / TP

[[7834  166]
 [ 665 7335]]


Classification Report

In [32]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8000
           1       0.98      0.92      0.95      8000

   micro avg       0.95      0.95      0.95     16000
   macro avg       0.95      0.95      0.95     16000
weighted avg       0.95      0.95      0.95     16000
 samples avg       0.95      0.95      0.95     16000



In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 96)        11712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 384)         885120    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 6, 256)        